In [ ]:
# 기본
import pandas as pd
import numpy as np
from functools import reduce

import xgboost as xgb
from sklearn.metrics import f1_score, classification_report
import matplotlib.pyplot as plt

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

### 파일패스 설정

In [ ]:
data_splits = ["train","test"]
months = ["07","08","09","10","11","12"]
# 각 데이터 유형별 폴더명, 파일 접미사, 변수 접두어 설정
data_categories = {
    "회원정보": {"folder": "1.회원정보", "suffix": "회원정보", "var_prefix": "customer"},
    "신용정보": {"folder": "2.신용정보", "suffix": "신용정보", "var_prefix": "credit"},
    "승인매출정보": {"folder": "3.승인매출정보", "suffix": "승인매출정보", "var_prefix": "sales"},
    "청구정보": {"folder": "4.청구입금정보", "suffix": "청구정보", "var_prefix": "billing"},
    "잔액정보": {"folder": "5.잔액정보", "suffix": "잔액정보", "var_prefix": "balance"},
    "채널정보": {"folder": "6.채널정보", "suffix": "채널정보", "var_prefix": "channel"},
    "마케팅정보": {"folder": "7.마케팅정보", "suffix": "마케팅정보", "var_prefix": "marketing"},
    "성과정보": {"folder": "8.성과정보", "suffix": "성과정보", "var_prefix": "performance"}
}

### ⚠️ 이 부분 꼭 확인해서 설정한 뒤에 실행!!
# local
root_path = '../data/open'

# colab
# root_path = '/content/drive/MyDrive/12조 파이널프로젝트/data'

In [ ]:
### ⚠️ 이 부분 꼭 확인해서 설정한 뒤에 실행!!
# 머신러닝 돌릴 월 선택 
select_month_num = 5

### 같은 월의 카테고리 전처리 데이터 통합

In [ ]:
# 모든 카테고리 돌면서 통합
for split in data_splits:
    merged_df=[]
    for category, info in data_categories.items():
        folder = info["folder"]
        suffix = info["suffix"]
        var_prefix = info["var_prefix"]
        
        file_path = f"{root_path}/{split}/{folder}/2018{months[select_month_num]}_processed_{suffix}.parquet"
        df = pd.read_parquet(file_path)

        ### 회원정보 카테고리가 아닌 경우, Segment 컬럼 다 지워주기
        if category == "회원정보":
            merged_df = df
        else:
            # 병합 (중복 컬럼 구분을 위해 suffixes 지정)
            merged_df = pd.merge(merged_df, df, on='ID', how='outer', suffixes=('', '_dup'))
            
            # 중복된 컬럼 제거
            for col in merged_df.columns:
                if col.endswith('_dup'):
                    base_col = col[:-4]  # _dup 제거
                    if base_col in merged_df.columns:
                        # base_col이 이미 있으면 중복된 _dup 컬럼은 삭제
                        merged_df.drop(columns=[col], inplace=True)
                    else:
                        # base_col이 없으면 _dup 컬럼을 base 이름으로 변경
                        merged_df.rename(columns={col: base_col}, inplace=True)
            print(f"🧹 {suffix} category 병합")
    
    # 저장
    output_file = f"{root_path}/{split}/2018{months[select_month_num]}_cleaned_All.parquet"
    merged_df.to_parquet(output_file, index=False)
    print(f"✅ 저장 완료: {split}/{months[select_month_num]}_cleaned_All")

### 통합된 파일 확인하기

In [ ]:
train_file = f"{root_path}/train/2018{months[select_month_num]}_cleaned_All.parquet"
df = pd.read_parquet(train_file)
df